In [27]:
import pandas as pd
import numpy as np
import cuvarbase.lombscargle as gls
import tqdm
import matplotlib.pyplot as plt
import os

os.environ['PATH'] = '/usr/local/cuda/bin'
os.environ['DYLD_LIBRARY_PATH'] = '/usr/local/cuda/lib'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib'

# Load train curves
df = pd.read_csv('data/training_set.csv',
                 dtype={
            'object_id': np.int32,
            'mjd': np.float32,
            'passband': np.int8,
            'flux': np.float32,
            'flux_err': np.float32,
            'detected': np.uint8,
        })

df_gb = df.groupby(['object_id', 'passband'])
uoids = df['object_id'].unique()
passbands = range(6)

In [28]:
grouped_ts = []

for oid in uoids[:10]:
    pb_group = []
    
    for pb in passbands:
        pb_group.append(
            tuple([df_gb.get_group((oid, pb))[series].values.astype('double') for series in ['mjd', 'flux', 'flux_err']])  
        )
        
    grouped_ts.append(pb_group)



In [31]:
# Set up LombScargleAsyncProcess (compilation, etc.)
proc = gls.LombScargleAsyncProcess()

# Run on single lightcurve
result = proc.run([grouped_ts[0]])

# Synchronize all cuda streams
proc.finish()

# Read result!
freqs, ls_power = result[0]

############
# Plotting #
############

f, ax = plt.subplots()
ax.set_xscale('log')

ax.plot(freqs, ls_power)
ax.set_xlabel('Frequency')
ax.set_ylabel('Lomb-Scargle')
plt.show()

CompileError: nvcc preprocessing of /tmp/tmpd9smix8r.cu failed
[command: nvcc --preprocess --use_fast_math -arch sm_61 -I/home/lanfear/anaconda3/envs/tf_intro/lib/python3.6/site-packages/pycuda/cuda /tmp/tmpd9smix8r.cu --compiler-options -P]
[stderr:
b'gcc: No such file or directory\n']